# **Pipeline Creation:**

## 1.Tokenizer

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
inputs = ('virat kohli is the best all formats batsmen i have seen in the world',
          'Bumrah is the best bowler in the world',
          'They both paly for team India')
out = tokenizer(inputs, padding=True, truncation = True, return_tensors='pt')
print(out)

{'input_ids': tensor([[  101,  6819,  8609, 12849, 27766,  2003,  1996,  2190,  2035, 11630,
         12236,  3549,  1045,  2031,  2464,  1999,  1996,  2088,   102],
        [  101, 26352, 10404,  2003,  1996,  2190, 14999,  1999,  1996,  2088,
           102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2027,  2119, 14412,  2100,  2005,  2136,  2634,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


## 2.Model Creation

In [ ]:
from transformers import AutoModel

In [ ]:
checkpoint='distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModel.from_pretrained(checkpoint)

In [ ]:
outputs = model(**out)
print(outputs.last_hidden_state.shape)

torch.Size([3, 19, 768])


## 3.Loading Into Model

In [ ]:
from transformers import AutoModelForSequenceClassification
result = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model_results = result(**out)

In [ ]:
print(model_results.logits.shape)

torch.Size([3, 2])


## 4.Post Processing the Output

In [ ]:
print(model_results.logits)

tensor([[-3.8897,  4.2887],
        [-4.0613,  4.4763],
        [-3.0931,  3.2366]], grad_fn=<AddmmBackward0>)


In [ ]:
import torch
predicted = torch.nn.functional.softmax(model_results.logits, dim=-1)
print(predicted)

tensor([[2.8057e-04, 9.9972e-01],
        [1.9592e-04, 9.9980e-01],
        [1.7794e-03, 9.9822e-01]], grad_fn=<SoftmaxBackward0>)


In [ ]:
result.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

# **Transformer Creation**

In [ ]:
#Initializing the model from the scratch
from transformers import BertModel, BertConfig
config = BertConfig()
model = BertModel(config)
print(config)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



we didn’t use BertConfig, and instead loaded a pretrained model via the bert-base-cased identifier. This is a model checkpoint that was trained by the authors of BERT themselves and it can also be fine-tuned on a new task. By training with pretrained weights rather than from scratch, we can quickly achieve good results.

In [ ]:
#Initializing the pretrained model with weights
from transformers import BertModel
model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
#we can also save the model for the future use
model.save_pretrained('My_Bert_Model')